In [21]:
import json
import numpy as np
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.geometry import shape
import pandas as pd

In [22]:
precincts=json.load(open('first-phase/Washington DC/WashingtonDC-precinct.geojson','r'))
# precincts=precincts['features']
# print(len(precincts))

polygons=[]
# precincts[0]
cx,cy=0,0
for precinct in precincts['features']:
  precinct['precinct']=precinct['properties']['precinct']
  polygons.append([shape(precinct['geometry']),precinct['properties']['precinct']])

long=0
lat=0
for p in polygons:
  cord=p[0].centroid.coords[0]
  long+=cord[0]
  lat+=cord[1]
long/=len(polygons)
lat/=len(polygons)
with open('./washington-city.json','w') as file:
  json.dump({
    "latitude": lat, 
    "longitude": long,
  },file)
# precincts['features']
# polygons[0][0].centroid.coords[0]

In [63]:
df=pd.read_csv('./first-phase/Washington DC/Washington DC Db.csv')
prev_crime_data=pd.read_csv('./wdc-precinct-wise-dataset.csv')
df=df[20001:30000]
list1=[]
for polygon in polygons:
  c=0
  count={'precinct': polygon[1],'total':0}
  # count=0
  for index,row in df.iterrows():
    if polygon[0].contains(Point(row['Longitude'],row['Latitude'])):
      if row['Offence'] not in count:
        count[row['Offence']]=0
      count[row['Offence']]+=1
      count['total']+=1
  list1.append(count)

In [67]:
# list1
crime_data=pd.DataFrame.from_dict(list1)
prev_crime_data=pd.read_csv('./wdc-precinct-wise-dataset.csv')
prev_crime_data=prev_crime_data.iloc[:,1:]
print(prev_crime_data.head())
print(crime_data.head())
# crime_data.to_csv('wdc-precinct-wise-dataset.csv')

   Arson  Assault  Murder  Robbery  Sex Crime  Theft  total  Precinct
0    0.0     12.0     2.0       89        2.0    278    383       401
1    0.0      1.0     0.0       20        0.0    107    128       201
2    0.0      7.0     2.0       60        1.0    210    280       406
3    0.0      9.0     2.0       49        3.0    181    244       402
4    0.0      5.0     0.0       62        3.0    218    288       202
   precinct  total  Theft  Robbery  Assault  Murder  Sex Crime  Arson
0       401    183    140       39      4.0     NaN        NaN    NaN
1       201     76     57       17      2.0     NaN        NaN    NaN
2       406    144    108       29      5.0     1.0        1.0    NaN
3       402    114     90       20      4.0     NaN        NaN    NaN
4       202    140    110       29      1.0     NaN        NaN    NaN


In [79]:
precincts=crime_data.iloc[:,0]
# print(precincts)
temp1=crime_data.iloc[:,1:].fillna(0)
temp2=prev_crime_data.iloc[:,:-1].fillna(0)
print(temp1.head())
print(temp2.head())
x=temp1.add(temp2)
x['precinct']=precincts
# x
x.to_csv('wdc-precinct-wise-dataset.csv')

   total  Theft  Robbery  Assault  Murder  Sex Crime  Arson
0    183    140       39      4.0     0.0        0.0    0.0
1     76     57       17      2.0     0.0        0.0    0.0
2    144    108       29      5.0     1.0        1.0    0.0
3    114     90       20      4.0     0.0        0.0    0.0
4    140    110       29      1.0     0.0        0.0    0.0
   Arson  Assault  Murder  Robbery  Sex Crime  Theft  total
0    0.0     12.0     2.0       89        2.0    278    383
1    0.0      1.0     0.0       20        0.0    107    128
2    0.0      7.0     2.0       60        1.0    210    280
3    0.0      9.0     2.0       49        3.0    181    244
4    0.0      5.0     0.0       62        3.0    218    288


In [5]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default='chrome'
crime_data=pd.read_csv('seattle-precinct-wise-dataset.csv')
data=json.load(open('./seattle.json'))
lat=data['latitude']
long=data['longitude']

In [6]:
fig=px.choropleth_mapbox(crime_data,locations='precinct',featureidkey="properties.precinct",geojson=precincts,color='total',color_continuous_scale='ylorrd')
# fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(mapbox_style="carto-positron", 
                  mapbox_zoom=9.5,
                  mapbox_center={"lat": lat, "lon": long},
                  margin={"r":0,"t":0,"l":0,"b":0},
                  uirevision='constant')
fig.write_html('first_figure-3.html')